# Funções

In [59]:
from filter_datasets import *
import pandas as pd
import json
import collections
import seaborn as sn
import matplotlib.pyplot as plt
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa
import glob
import urllib.parse
import urllib.request
import uniprot
import requests
from Bio import SeqIO
from io import StringIO
from pandas_profiling import ProfileReport

In [5]:
def download_projects(project_list):
    """
        Download all mztab files as well as creates a directory with all files available for download
        Parameters:
            project_list: list of projects to download 
    """
    for projects in project_list:
        print(f"Downloading projetc:{projects}") 
        pride_obj = ppx.PrideProject(
        projects,
        local = os.path.join(os.getcwd(),mztab_storage_folder,f"{projects}")
        )
        pride_obj.download(pride_plants_mztab[projects])
        


In [6]:
def unzip_mztabs(path):
    """
    Unzip all mztba.gz files
    """
    for proj_dir,subdir,files in os.walk(os.path.join(os.getcwd(),mztab_storage_folder)):
        if proj_dir.endswith("/generated"):
           os.system(f"gunzip {os.path.join(proj_dir,'*.gz')}")


In [7]:
def get_allmztab(path):
    """
    Get a list with all mztab files 
    """
    all_files = glob.glob(path)
    return all_files

In [8]:
# usar accession de PSM para pegar description em PRT
def PSM2PRT (mztab,accession):
    """
    Uses mztab["PRT"] accession to get pep sequence description availabel only on mztab["PSM"]
    Parameters:
        mztab: file to search
        accession: accession number
    """   
    for pst_data in mztab["PRT"].itertuples():
            if pst_data.accession == accession:
                return pst_data.description


In [9]:
def resume_mztab(file_list:list,specie ):
    """
        Returns a fasta file, per project, with all peps on all mztabs
        Parameters:
            file_list: list containg mztab files
            specie: specie to name the returning fasta file   
    """
    for files in file_list:
        mztab_obj = mztab.MzTab(files)
        projetc_number  = mztab_obj.metadata["mzTab-ID"]
        with open(f"{specie}_{projetc_number}.fasta","w+") as file_fasta:
            for info in mztab_obj["PSM"].itertuples():
                file_fasta.write(f">{info.accession} | {PSM2PRT(mztab_obj,info.accession)} \n") # Se possível, preciso melhorar isso. O(n²) é lento
                file_fasta.write(f"{info.sequence}\n") 
    return file_fasta

In [10]:
def get_swissprot_sequences(accession:list,out_file_name:str,project):
    """
    convert pride accession number to swissprot accession number then downloads complete protein sequences
        Parameters:
            accession: List containg PRIDE IDs

    """
    databases_url = 'https://www.uniprot.org/uploadlists/'
    query = None
    if len(accession) == 1:
        query = accession
        params = {
        'from': 'ACC+ID', #pride accession
        'to': 'SWISSPROT', #swissprot accession
        'format': 'list',
        'query': f'{query}'}
    else:
        params = {
        'from': 'ACC+ID', #pride accession
        'to': 'SWISSPROT', #swissprot accession
        'format': 'list',
        'query': f'{" ".join(accession)}'}
    pride_accession = urllib.parse.urlencode(params)
    pride_accession = pride_accession.encode('utf-8')
    req = urllib.request.Request(databases_url, pride_accession)
    with urllib.request.urlopen(req) as accession_file:
        response = accession_file.read()
    swissprot_accession = list(response.decode('utf-8').split("\n"))

    
    # Get protein sequences from swissprot
    swissprot_url = "http://www.uniprot.org/uniprot/"
    with open(f"{'_'.join(out_file_name)}_{project}.complete.fasta","w+") as fasta:
        for IDs in swissprot_accession:
            joint_url = swissprot_url + IDs + ".fasta"
            swissprot_response = requests.post(joint_url)
            raw_data = "".join(swissprot_response.text)
            Seq = StringIO(raw_data)
            for seq_info in SeqIO.parse(Seq,'fasta'):
                fasta.write(f">{seq_info.description}\n")
                fasta.write(f"{seq_info.seq}\n")
    return fasta

In [11]:
def concatenate_swissprot(file_list, specie):
    """
    Concatenate get_swissprot_sequences results
    """
    for files in file_list:
        mztab_obj = mztab.MzTab(files)
        project_number  = mztab_obj.metadata["mzTab-ID"]
        accession_list = mztab_obj["PRT"].accession.values
        get_swissprot_sequences(accession_list,specie,project_number)

In [12]:
def download_sequences(mztab_prt,out_file):
    """ """
    project_id  = mztab_prt.metadata["mzTab-ID"]
    with open(f"{os.path.join(out_file,project_id)}.fasta","w+") as sequence_fasta_file:
        for data in mztab_prt["PRT"].itertuples():
            uniprot_url = 'https://www.uniprot.org/uploadlists/'
            params = {
            'from': 'ACC+ID', #swissprot accession
            'to': 'EMBL', #ENA accession
            'format': 'list',
            'query': f'{data.accession}'
            }
            pride_accession = urllib.parse.urlencode(params)
            pride_accession = pride_accession.encode('utf-8')
            pride_request = urllib.request.Request(uniprot_url, pride_accession)
            with urllib.request.urlopen(pride_request) as request_data:
                pride_request_response = request_data.read()
                
            ENA_accession = pride_request_response.decode("utf-8").split('\n')
            ENA_ulr = url = f"https://www.ebi.ac.uk/ena/browser/api/fasta/{ENA_accession[0]}?lineLimit=1000"
            ENA_seq_request = requests.get(url).text.split("\n")
            fasta_id = data.description
            fasta_seq = "".join(ENA_seq_request[1:])
            sequence_fasta_file.write(f">{fasta_id}\n")
            sequence_fasta_file.write(f"{fasta_seq}\n")

# 1. Objetivos
**Geral**: encontrar RNA de plantas tidos como não codificantes mas que, na verdade, codificam proteínas

**Específico**: Varrer os bancos de dados de proteômica em busca de proteínas candidatas a serem codificadas por mRNAs tidos como não codificantes


### 1.1. PRoteomcis IDEntifications Database (PRIDE) [link](https://www.ebi.ac.uk/pride/)

Visão geral do PRIDE

In [13]:
# pride_plant.json foi gerado usando a função filterPride para checar o número de projetos com plantas 
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total de projetos com plantas no PRIDE: {len(pride_plant)}")

Total de projetos com plantas no PRIDE: 1464


In [14]:
pd.DataFrame(pride_plant,).T.head()

,name,accession
PRD000037,Chlamydomonas reinhardtii,3055
PRD000044,Arabidopsis thaliana (mouse-ear cress),3702
PRD000051,Zea mays (maize),4577
PRD000084,Arabidopsis thaliana (mouse-ear cress),3702
PRD000096,Arabidopsis thaliana (mouse-ear cress),3702


In [15]:
# Pride_plants_mztab.json foi gerado usando a função get_mztabes. Ele contem os projetos com plantas e com arquivos mztab
with open("pride_plants_mztab.json") as mztabs:
    pride_plants_mztab = json.load(mztabs)
print(f"Total de projetos com arquivos mztab: {len(pride_plants_mztab)}")

Total de projetos com arquivos mztab: 270


In [16]:
def get_projects(dict_plants = pride_plant, dict_mztab = pride_plants_mztab, specie = None):
    """
    Returns all projecst with mztab files for a give plant specie. Search by name
    parameters:
        dict_plants: contains all plant projects retrivied from PRIDE
        pride_plants_mztab: contains all plant projects with mztab retrivied from PRIDE
        specie: specie's name to search
    """
    projetcs_total = [projects for projects,values in dict_plants.items() if f"{specie}" in values["name"]]
    projects = list(set(dict_mztab.keys()) & set(projetcs_total))
    return projects

Arquivos mztab por espécie

In [17]:
pride_plants_names = [values["name"] for keys,values in pride_plant.items()]
pride_plants_counter = collections.Counter(pride_plants_names)
plant_names = [plants[0] for plants in pride_plants_counter.items()]
plant_quant = [plants[1] for plants in pride_plants_counter.items()]
plant_with_mztab = [] 
for specie in pride_plants_counter.keys():
    all_projects = [projects for projects,values in pride_plant.items() if f"{specie}" == values["name"]]
    mztab_projects = list(set(pride_plants_mztab.keys()) & set(all_projects))
    plant_with_mztab.append(len(mztab_projects))
plants_count_dataframe = pd.DataFrame({
    "Specie":plant_names,
    "Total_projects":plant_quant,
    "Projects_with_mztab":plant_with_mztab
})
plants_count_dataframe.sort_values("Total_projects", ascending = False, inplace = True)


In [18]:
plants_count_dataframe

,Specie,Total_projects,Projects_with_mztab
1,Arabidopsis thaliana (mouse-ear cress),490,110
45,Triticum aestivum (wheat),73,11
18,Rice(o. sativa),58,10
2,Zea mays (maize),57,7
0,Chlamydomonas reinhardtii,50,12
...,...,...,...
129,Micromonas sp. (strain rcc299 / noum17) (picop...,1,1
130,Zingiber officinale,1,0
133,Leymus chinensis,1,0
134,Cannabis,1,1


In [62]:
plt.figure(figsize = (12,8))
projects = plt.barh(plants_count_dataframe.Specie[:20],plants_count_dataframe.Total_projects[:20],color = 'b',align='center', alpha = .6)
mztabs = plt.barh(plants_count_dataframe.Specie[:20],plants_count_dataframe.Projects_with_mztab[:20],color = 'g',align='center', alpha = 0.6,)
plt.set_ylabel("Espécie",fontsize = 20)
plt.set_xlabel("Número de projetos",fontsize = 20)
plt.set_title("20 plantas mais comuns (PRIDE)", fontsize = 30)
plt.tick_params(labelsize = 18)
plt.legend((projects,mztabs),("Total de projetos","Projetos com arquivos mztab"),fontsize = 20)
plt.savefig("/home/tiago/documents/lncRNA/figures/most_commum.png")


AttributeError: module 'matplotlib.pyplot' has no attribute 'set_ylabel'

### 3. Caminhos possíveis 

1. Os arquivos do PRIDE contem peptídeos que fazem parte de proteínas não caracterizadas e que podem ser mapeadas de volta em regiões codificam RNAs não codificantes nos genomas das espécias alvo.

2. Os arquivos do PRIDE contem peptídeos qua não estão nos proteoma das espécies alvo.


![Fluxograma](figures/fluxograma.jpg)

# Resultados

### *Coffea arabica*

Criar diretório onde os arquivos e resultados serão armazenados

In [18]:
import os
mztab_storage_folder = "C.arabica2"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

In [ ]:
#Dowload all gene sequences of proteome in PRIde mztab files
specie = "Coffea arabica"
projects = get_projects(specie = specie) # Find PRIDE projects for a given specie
download_projects(projects) #Download all projects 
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder))
for proj_dir,subdir,files in os.walk(os.path.join(os.getcwd(),mztab_storage_folder)):
    if proj_dir.endswith("/generated"):
        all_mztab_files = get_allmztab(os.path.join(proj_dir,"*.mztab")) #Concatenate all mztabs by projetc
        for mztab_path in all_mztab_files:
            mztab_obj = mztab.MzTab(mztab_path) #Create mztab object
            print(proj_dir.split("/")[-2])
            download_sequences(mztab_obj,proj_dir) #Crossref between Uniprot/Swissprot and ncbi

## Em andamento

### Soy bean

In [2]:
mztab_storage_folder = "G.max"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

Folder already exist: /home/tiago/documents/lncRNA/G.max


In [ ]:
specie = "Glycine max"
projects = get_projects(specie = specie)
print(f"Total of projects for {specie}:{len(projects)}")
download_projects(projects)

In [53]:
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder))

In [16]:
unzip_mztabs("G.max/PXD000201/generated")

Projeto teste PXD000201

### *Arabidopsis thaliana*

In [48]:
mztab_storage_folder = "A.thaliana"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

Folder already exist: /home/tiago/documents/lncRNA/A.thaliana


In [45]:
specie = "Arabidopsis thaliana"
projects = get_projects(specie = specie)
download_projects(projects)
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder))

TOTAL:   0%|          | 0/8 [00:00<?, ?files/s]

generated/PRIDE_Exp_Complete_Ac_9886.pride.mztab.gz:   0%|          | 0.00/438k [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_9887.pride.mztab.gz:   0%|          | 0.00/418k [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_9888.pride.mztab.gz:   0%|          | 0.00/84.0k [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_9889.pride.mztab.gz:   0%|          | 0.00/89.4k [00:00<?, ?b/s]

...

generated/Velos_T5.pride.mztab.gz:   0%|          | 0.00/10.8k [00:00<?, ?b/s]

generated/Velos_T6.pride.mztab.gz:   0%|          | 0.00/16.4k [00:00<?, ?b/s]

generated/Velos_T7.pride.mztab.gz:   0%|          | 0.00/7.36k [00:00<?, ?b/s]

generated/Velos_T8.pride.mztab.gz:   0%|          | 0.00/8.00k [00:00<?, ?b/s]

generated/Velos_T9.pride.mztab.gz:   0%|          | 0.00/13.3k [00:00<?, ?b/s]